In [1]:
# modules
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
# Importing core neural networks layers 
from keras.layers import Dense, Dropout, Activation, Flatten, UpSampling2D
# Importing CNN layers
from keras.layers import Conv2D, MaxPooling2D, Input
# Importing the sequential model in Keras
# Used as a linear stack of nn layers
from keras.models import Sequential, Model
# Tools for data transformation
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.applications import VGG16
from keras.optimizers import RMSprop
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

Using TensorFlow backend.


# 3 Text Classification
While images contain local spatial correlations and structure, many other datasets contain temporal correlations. Examples include time series and discrete se- quences such as text. In this problem, we will tackle the task of text classifica- tion in the context of cybersecurity.<br/>
&nbsp;&nbsp;<b>Background.</b> When malware infects a host computer, it often needs to reach out to an outside server for further instructions or to download additional payloads. This outside server is called a Command-and-Control server (C2). The malware needs to send a specific communication to the C2 server, thus the C2 server needs to have a registered IP address or associated web domain so that it can be reached. Therefore, being able to identify web domains that are likely related to malware C2 can be a valuable cyber defense.<br/>
&nbsp;&nbsp;<b>Dataset.</b> Fortunately, security researchers have already identified and logged a large number of malicious URLs. Additionally, we can catalog common ”be- nign” URLs just from typical web behavior (these would include things like facebook.com and amazon.com). Hence, we have a labeled dataset for text classification which can be downloaded here:
<br/>• https://s3.amazonaws.com/anly-590/url-classification/benign-urls.txt
<br/>• https://s3.amazonaws.com/anly-590/url-classification/malicious-urls.txt
## 3.1 RNN
Build and train a Recurrent Neural Network to solve this text classification task. You can use any type of RNN you wish (SimpleRNN, GRU, LSTM).
## 3.2 CNN
Build and train a 1D CNN for this text classification task. You might gain some insight and inspiration from these text classification approaches:
<br/>• http://www.aclweb.org/anthology/D14-1181 
<br/>• https://arxiv.org/abs/1702.08568
## 3.3
Be sure to directly compare your two methods with an ROC curve or similar validation method. Don’t forget to create a train-test split.

In [3]:
# 3.1 RNN

# Load data-sets
def read_file( file_name ):
    with open( file_name ) as file:
        lines = file.readlines()
    return [ line.rstrip("\n\t") for line in lines[ 0: ] ]

mal_urls = read_file( 'malicious-urls.txt' )
ben_urls = read_file( 'benign-urls.txt' )

IOError: [Errno 2] No such file or directory: 'malicious-urls.txt'